In [ ]:
!pip install -r /home/soroush1/projects/def-kohitij/soroush1/idiosyncrasy/requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
script_dir = os.path.dirname(os.getcwd())  # Get the directory where the script is located
parent_dir = os.path.dirname(script_dir)  # Get the parent directory

print(f"{script_dir = }")
print(f"{parent_dir = }")

sys.path.append(parent_dir)

script_dir = '/lustre06/project/6067616/soroush1/idiosyncrasy/notebooks'
parent_dir = '/lustre06/project/6067616/soroush1/idiosyncrasy'


In [14]:
from lit_modules.datamodule import MuriDataModule
from argparse import Namespace
from datasets.Muri import MuriDataset

In [ ]:
hparams = Namespace(
    data_dir="/scratch/soroush1/memorability/muri1320",
    image_size=224,
    batch_size=128,
    num_workers=4,
    change_labels = False,
    pin_memories=[False, False, False],  # [train, val, test]
    return_paths = True
)

# Create the DataModule
data_module = MuriDataModule(hparams)

# Prepare data and setup
data_module.prepare_data()
data_module.setup("test")

# Print dataset sizes
print(f"test dataset size: {len(data_module.test_dataset)}")

test_dl = data_module.test_dataloader()

In [4]:
from datasets.Muri import MuriDataset

ds = MuriDataset(root="/scratch/soroush1/memorability/muri1320")

In [47]:
import os
import os
from torch.utils.data import Dataset
from torchvision import transforms as T
import PIL.Image
import re
import pandas as pd
from glob import glob
import torch
from torchvision import transforms


class MuriDataset(Dataset):
    def __init__(self, root: str, transforms=None, return_paths=False):

        self.root = root

        self.transforms = transforms
        # Sort image files
        self.meta_data = pd.read_csv(os.path.join(root, "meta.csv"))

    def __len__(self):
        return len(self.meta_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.meta_data["img_path"][idx]
        print(f"{img_path = }")
        image = PIL.Image.open(img_path).convert("RGB")

        if self.transforms:
            image = self.transforms(image)
            
        # Get the object label
        label = self.meta_data["labels"][idx]

        return image, label

def val_transform(input_size: int = 256):
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
    )

    return transforms.Compose(
        [
            transforms.Resize(input_size),
            transforms.ToTensor(),
            normalize,
        ]
    )

In [1]:
input_size = 256
ds = MuriDataset(root="/scratch/soroush1/memorability/muri1320", transforms=val_transform(input_size=input_size))
print(f"{len(ds) = }")
x, y = ds[0]
x.size(), y

NameError: name 'MuriDataset' is not defined

In [69]:
df = pd.read_csv("/scratch/soroush1/memorability/muri1320/meta.csv")

# How many of the labels are in the dataset?
# use groupby to count the number of unique labels
# then print the dataframe

df.groupby("labels").size().reset_index(name="counts")

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.5, random_state=42, stratify=df["labels"]) 

# save index of train_df and test_df
train_df["Unnamed: 0"].to_csv("/scratch/soroush1/memorability/muri1320/train.csv", index=False)
test_df["Unnamed: 0"].to_csv("/scratch/soroush1/memorability/muri1320/test.csv", index=False)